In [ ]:
from pathlib import Path
import albumentations as A
from sklearn.model_selection import train_test_split
from fastai.vision.all import *

In [ ]:
SEED = 42
VAL_RATIO = 0.2
IMAGE_SIZE = 224
DATASET_ROOT = Path("dog-emotions-prediction")
IMAGE_ROOT = DATASET_ROOT / Path("images")

In [ ]:
def set_seed(x=42):
    random.seed(x)
    np.random.seed(x)
    torch.manual_seed(x)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(x)

set_seed(SEED)

In [ ]:
def cusTrainTestSplitter(
    test_size=0.2, random_state=None, stratify=False, train_size=None, shuffle=True
):
    "Split `items` into random train and test subsets using sklearn train_test_split utility."

    def _inner(o, **kwargs):
        o = list(o)
        labels = [i.parent.name for i in o]
        train, valid = train_test_split(
            range_of(o),
            test_size=test_size,
            random_state=random_state,
            stratify=labels if not stratify else None,
            train_size=train_size,
            shuffle=shuffle,
        )
        return L(train), L(valid)

    return _inner


class AlbumentationsTransform(DisplayedTransform):
    split_idx, order = 0, 2

    def __init__(self, train_aug):
        store_attr()

    def encodes(self, img: PILImage):
        aug_img = self.train_aug(image=np.array(img))["image"]
        return PILImage.create(aug_img)


def get_train_aug():
    return A.Compose(
        [
            A.HueSaturationValue(
                hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
            ),
            A.CoarseDropout(p=0.5),
            A.Cutout(p=0.5),
        ]
    )


dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=cusTrainTestSplitter(
        test_size=VAL_RATIO, random_state=SEED, shuffle=True, stratify=True
    ),
    get_y=lambda x: x.parent.name,
    item_tfms=[AlbumentationsTransform(get_train_aug()), Resize(IMAGE_SIZE), ToTensor],
    batch_tfms=[IntToFloatTensor, Normalize.from_stats(*imagenet_stats)],
)

In [ ]:
dls = dblock.dataloaders(IMAGE_ROOT)
learn = vision_learner(dls,"resnet34",loss_func=CrossEntropyLossFlat(),opt_func=Adam,metrics=F1Score(average='macro'))
learn.fine_tune(10)